In [1]:
import LDpred as ld
import numpy as np

pwd = "/Users/yuanwen/Desktop/Docker_Environment/intern/1"
precision_folder_path = pwd + "/data/EUR"
snplist_folder_path = pwd + "/data/snplists_GRCh38positions"
sumstats_path = pwd + "/data/sumstats/public-data3-sumstats.txt"
vcf_path = pwd + "/data/sumstats/public-data3_vcf.vcf"
fam_path = pwd + "/data/sumstats/public-data3.fam"

PM = ld.Read.PM_read(precision_folder_path)
snplist = ld.Read.snplist_read(snplist_folder_path)
sumstats = ld.Read.sumstats_read(sumstats_path)
ld.Fliter.filter_by_PM(PM, snplist)
sumstats_set = ld.Fliter.fliter_by_sumstats(PM, snplist, sumstats)
para = {"h2": 0.2966, "p": 0.018, "burn_in": 50, "num_iter": 100, "N": 150000}
sumstats_simu, beta_true = ld.generate.generate_sumstats_beta(PM, snplist, para)
beta_grid = ld.LDpred2.ldpred2_grid(PM, sumstats_simu, para)
beta_inf = ld.LDpred2.ldpred2_inf(PM, sumstats_simu, para)
ld.predict.check_h2(PM, beta_true), ld.predict.check_h2(PM, beta_grid), ld.predict.check_h2(PM, beta_inf)



# vcfstats = ld.Read.vcf_read(vcf_path)
# famstats = ld.Read.fam_read(fam_path)
# phestats = ld.Fliter.merge_vcf_fam(vcfstats, famstats)
# phestats_total = ld.Fliter.fliter_by_vcf(beta_grid, snplist, phestats)

# ld.Fliter.fliter_by_REF_ALT(snplist, phestats_total)
# ld.predict.normalize(phestats_total)
# ld.predict.check_r_squared(phestats_total, beta_grid_total)

/Users/yuanwen/Desktop/Docker_Environment/intern/1/code/snp_read/Fliter.py:33: SparseEfficiencyWarning: splu converted its input to CSC format
  splu(PM[i]["precision"]).solve(I_dense[:, rsid])[rsid]
/Users/yuanwen/Desktop/Docker_Environment/intern/1/code/simulation/generate.py:29: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  sumstats_block["beta"] = np.random.multivariate_normal(


(0.293947224855556, 0.27832714599614394, 0.14871343110277452)

In [21]:
def generate_sumstats_beta(PM, snplist, para):
    sumstats = []
    beta_true = []
    m = 0
    for i in range(len(snplist)):
        m += len(snplist[i]["rsid"])

    sigma = para["h2"] / (m * para["p"])
    for i in range(len(snplist)):
        sumstats_block = {}
        mm = len(snplist[i]["rsid"])
        sumstats_block["N"] = np.ones(mm) * para["N"]
        sumstats_block["beta_se"] = np.ones(mm) * np.sqrt(sigma)
        beta_true.append([])
        if mm == 0:
            sumstats_block["beta"] = []
            sumstats.append(sumstats_block)
            continue
        for j in range(mm):
            if np.random.rand() < para["p"]:
                beta_true[i].append(np.random.normal(0, np.sqrt(sigma)))
            else:
                beta_true[i].append(0)
        beta_true[i] = np.array(beta_true[i])
        R = PM[i]["LD"].toarray()
        sumstats_block["beta"] = np.random.multivariate_normal(
            R @ beta_true[i], R / para["N"]
        )
        sumstats.append(sumstats_block)

    return sumstats, beta_true
para = {"h2": 0.2966, "p": 0.018, "burn_in": 50, "num_iter": 100, "N": 30000}
sumstats_simu, beta_true = generate_sumstats_beta(PM, snplist, para)
beta_grid = ldpred2_grid(PM, sumstats_simu, para)
beta_inf = ldpred2_inf(PM, sumstats_simu, para)
ld.predict.check_h2(PM, beta_true), ld.predict.check_h2(PM, beta_grid), ld.predict.check_h2(PM, beta_inf)

/var/folders/8x/dw7m7l3j279g44xv2n3fsmx40000gn/T/ipykernel_54969/3637932132.py:26: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  sumstats_block["beta"] = np.random.multivariate_normal(


(0.28155731466395395, 0.19748235869165198, 0.0494852815843795)

In [9]:
def check_h2(PM, beta):
    h2 = 0
    for i in range(len(PM)):
        h2 += np.dot(beta[i], PM[i]["LD"].toarray() @ beta[i])
    return h2
check_h2(PM, beta_true)

0.29916655961911187

In [56]:
LD = PM[0]["LD"]
print(LD[:, 3].toarray().flatten())

[-1.25452191e-11 -5.21323544e-11  1.40438052e-09  1.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]


In [19]:
import numpy as np
from scipy.sparse.linalg import inv, spsolve
from scipy.sparse import csr_matrix, eye


def get_marginal_beta(sumstats):
    beta_marginal = []
    for i in range(len(sumstats)):
        beta_marginal.append(list(map(float, sumstats[i]["beta"])))
    return beta_marginal


def ldpred2_inf(PM, sumstats, para):
    beta_inf_set = []
    m = 0
    for i in range(len(sumstats)):
        m += len(sumstats[i]["N"])
    for i in range(len(PM)):
        if len(sumstats[i]["N"]) == 0:
            beta_inf_set.append([])
            continue
        N = np.array(sumstats[i]["N"]).astype(float)
        beta_se = np.array(sumstats[i]["beta_se"]).astype(float)
        beta = np.array(sumstats[i]["beta"]).astype(float)
        scale = np.sqrt(N * beta_se**2 + beta**2)
        beta_hat = beta / scale
        LD = PM[i]["LD"]
        LD += eye(LD.shape[0], format="csr") * (m / (para["h2"] * para["N"]))
        beta_inf = spsolve(LD, beta_hat)
        beta_inf_set.append(beta_inf * scale)
    return beta_inf_set


def ldpred2_gibbs_one_sampling(PM, beta_hat, N, M, para):
    LD = PM["LD"]
    # LD = inv(PM["precision"])
    if isinstance(LD[0], np.float64):
        LD = csr_matrix([[LD[0]]])
    m = len(beta_hat)
    h2 = para["h2"]
    p = para["p"]
    curr_beta = np.zeros(m)
    avg_beta = np.zeros(m)
    dotprods = np.zeros(m)
    h2_per_var = h2 / (M * p)
    inv_odd_p = (1 - p) / p
    for k in range(-para["burn_in"], para["num_iter"]):
        for j in range(m):
            # res_beta_hat_j = beta_hat[j] - (dotprods[j] - curr_beta[j])
            res_beta_hat_j = beta_hat[j] - dotprods[j]
            C1 = h2_per_var * N[j]
            C2 = 1 / (1 + 1 / C1)
            C3 = C2 * res_beta_hat_j
            C4 = C2 / N[j]
            post_p_j = 1 / (1 + inv_odd_p * np.sqrt(1 + C1) * np.exp(-C3 * C3 / C4 / 2))
            diff = -curr_beta[j]
            # if post_p_j < p:
            #     curr_beta[j] = 0
            if post_p_j > np.random.rand():
                curr_beta[j] = np.random.normal(C3, np.sqrt(C4))
                diff += curr_beta[j]
            else:
                curr_beta[j] = 0
            if k >= 0:
                avg_beta[j] += C3 * post_p_j
        if diff != 0:
            dotprods += LD[:, j].toarray().flatten() * diff
    return avg_beta / para["num_iter"]


def ldpred2_grid(PM, sumstats, para):
    beta_gibbs_set = []
    M = 0
    for i in range(len(sumstats)):
        M += len(sumstats[i]["beta"])
    for i in range(len(PM)):
        if len(sumstats[i]["beta"]) == 0:
            beta_gibbs_set.append([])
            continue
        N = np.array(sumstats[i]["N"]).astype(float)
        beta_se = np.array(sumstats[i]["beta_se"]).astype(float)
        beta = np.array(sumstats[i]["beta"]).astype(float)
        # scale = np.sqrt(N * beta_se**2 + beta**2)
        scale = 1
        beta_hat = beta / scale
        beta_gibbs = ldpred2_gibbs_one_sampling(PM[i], beta_hat, N, M, para)
        beta_gibbs_set.append(beta_gibbs * scale)
    return beta_gibbs_set
